In [18]:
import pymongo

def create_connection():
    try:
        client = pymongo.MongoClient("mongodb://localhost:27017/")
        return client["IUM-Calcio"], client
    except pymongo.errors.ConnectionFailure as e:
        print(f"Connection failed: {e}")
        return None, None


def close_connection(client):
    # Close connection
    if client is not None:  # Check for None explicitly
        client.close()
 

In [60]:
def retrieve(db, subject, stats, subject_info, subject_info_key, initial_year, final_year):
    if subject == "players":
        try:
            main_collection = db["players"]
            collection_events = db["game_events"]
            pipeline = [
                        {
                           "$lookup": {
                                 "from": "game_events", 
                                 "localField": "player_id", 
                                 "foreignField": "player_id", 
                                 "as": "player_events",
                            }
                        },
                        {
                            "$match": {
                                "player_id": 4425 #4425
                            }
                        },
                        {
                            "$unwind": "$player_events"
                        },
                        {
                            "$match": {
                                "player_events.type": "Goals"
                            }
                        },
                        {
                            "$group": {
                                "_id": "$player_id",
                                "total_goles": {"$sum": 1}
                            }
                        },
                        {
                          "$project": {
                              "player_id": 1,
                              "total_goles": 1,
                          }  
                        },
                        
            ]
            resultado_agregacion = list(main_collection.aggregate(pipeline))

            return resultado_agregacion
            #query_clubs = {subject_info: subject_info_key}
            #projection = {"goals": stats[0], "assists": stats[1], "cards": stats[2]}
            #old_players_clubs = list(main_collection.aggregate(query_clubs, projection, pipeline))
            #return old_players_clubs
        except Exception as e:
            print(f"Retrieval of Clubs failed: {e}")
            return []
    else:
        return []

db, client = create_connection()

In [61]:
result = retrieve(db,"players",(1,0,0),"country_of_citizenship", "Germany", 2012,2012)
for x in result:
    print(x)

{'_id': 4425, 'total_goles': 6}


The stats will be about a subject chosen between:
 - "players"
 - "clubs"
 - "competitions"

In [7]:
subject = "players"

The user defines the stats that wants to see. Stats is a tuple of 3 integer elements, always in the next order:
(Goals, Assists, Cards)
Here are some examples of how the tuple could be in different cases:
If the user wants to see the only goals and assists, the value of the tuple will be (1,1,0).
If the user wants to see the only goals and cards, the value of the tuple will be (1,0,1).
If the user wants to see the only goals, the value of the tuple will be (1,0,0).
If the user wants to see those three assists, the value of the tuple will be (1,1,1).

In [8]:
stats = (1,0,1)

Choose the information of the subject to search for stats, the options are:
 - "name"
 - "country"

In [9]:
subject_info = "name"

Once chosen, indicate here the name of the subject in case of "name", or the country in case of "country".

In [10]:
subject_info_key = "John"

Last, it will be chosen the period of years in which the user want to get stats. If the user only wants the stats of one year, "initial_year" and "final_year" will have the same value. The range of years goes from 2012 to 2023.

In [11]:
initial_year = 2012
final_year = 2020